In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import datetime
import tqdm
import re
import time
from string import punctuation
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
from sklearn.model_selection import train_test_split

import torch, torch.nn as nn
import torch.nn.functional as F

sns.set_style('darkgrid')
mpl.rcParams['figure.figsize'] = [15,10]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from kaggle.competitions import nflrush

# Utils

In [ ]:
import pandas as pd
import numpy as np
from string import punctuation
import datetime
import re

def crps(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred), axis=1)

def yard_to_cdf(yard):
    yard = np.round(yard).astype(int)
    indices = yard+99
    cdfs = np.zeros((yard.shape[0], 199))
    for i in range(len(cdfs)):
        cdfs[i, indices[i]:] = 1
    return cdfs

def cdf_to_yard(cdf):
    yard_index = (cdf==1).argmax(axis=1)
    yard = yard_index-99
    return yard


def cdf_to_yard_torch(cdf):
    yard_index = torch.sum((torch.as_tensor(cdf) <= 0),dim=1)
    yard = yard_index-99
    return yard

def crps_torch(y_true, y_pred):
    y_true = torch.as_tensor(y_true)
    y_pred = torch.as_tensor(y_pred)
    return torch.mean((y_true - y_pred).pow(2), dim=1)

def crps_loss(y_true, y_pred_pdf):
    y_pred_cdf = torch.cumsum(torch.as_tensor(y_pred_pdf), dim=1)
    return crps_torch(y_true, y_pred_cdf).mean()

def crps_loss_cdf(y_true, y_pred_cdf):
    return crps_torch(y_true, y_pred_cdf).mean()
    
def clean_StadiumType(txt):
    if pd.isna(txt):
        return np.nan
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    txt = txt.replace('outside', 'outdoor')
    txt = txt.replace('outdor', 'outdoor')
    txt = txt.replace('outddors', 'outdoor')
    txt = txt.replace('outdoors', 'outdoor')
    txt = txt.replace('oudoor', 'outdoor')
    txt = txt.replace('indoors', 'indoor')
    txt = txt.replace('ourdoor', 'outdoor')
    txt = txt.replace('retractable', 'rtr.')
    return txt

def transform_StadiumType(txt):
    if pd.isna(txt):
        return np.nan
    if 'outdoor' in txt or 'open' in txt:
        return 1
    if 'indoor' in txt or 'closed' in txt:
        return 0
    return np.nan

def str_to_seconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans

def str_to_float(txt):
    try:
        return float(txt)
    except Exception as e:
        return np.NaN

def map_weather(txt):
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

def new_orientation(angle, play_direction):
    if play_direction == 0:
        new_angle = 360.0 - angle
        if new_angle == 360.0:
            new_angle = 0.0
        return new_angle
    else:
        return angle

def preprocess_features(df, fillna=False):
    """Accepts df like train data, returns X, y"""

    # Feature engineering
    df['DefendersInTheBox_vs_Distance'] = (df['DefendersInTheBox'] / df['Distance'])
    df['StadiumType'] = df['StadiumType'].apply(clean_StadiumType)
    df['StadiumTypeShort'] = df['StadiumType'].apply(transform_StadiumType)
    df = df.drop(['StadiumType'], axis=1)

    Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 
        'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 
        'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 
        'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 
        'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 

    turf_type = df['Turf'].map(Turf)
    df['TurfIsNatural'] = (turf_type == 'Natural')
    df = df.drop(['Turf'], axis=1)

    # CAREFUL. What if a new team appears?
    map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
    for abb in df['PossessionTeam'].unique():
        map_abbr[abb] = abb

    def safe_map(val):
        if map_abbr.get('val'):
            return map_abbr[val]
        else:
            return val

    df['PossessionTeam'] = df['PossessionTeam'].apply(safe_map)
    df['HomeTeamAbbr'] = df['HomeTeamAbbr'].apply(safe_map)
    df['VisitorTeamAbbr'] = df['VisitorTeamAbbr'].apply(safe_map)

    df['HomePossesion'] = (df['PossessionTeam'] == df['HomeTeamAbbr'])

    df['Field_eq_Possession'] = df['FieldPosition'] == df['PossessionTeam']
    df['HomeField'] = df['FieldPosition'] == df['HomeTeamAbbr']

    # in posession
    df['InPosession']=(((df.Team == 'home') & (df.PossessionTeam == df.HomeTeamAbbr)) | ((df.Team == 'away') & (df.PossessionTeam == df.VisitorTeamAbbr)))

    # Formation columns
    df = pd.concat([df.drop(['OffenseFormation'], axis=1), pd.get_dummies(df['OffenseFormation'], prefix='OffenseFormation')], axis=1)
    # Filling missing dummy columns at test stage
    expected_columns = ['OffenseFormation_ACE',
         'OffenseFormation_EMPTY',
         'OffenseFormation_JUMBO',
         'OffenseFormation_PISTOL',
         'OffenseFormation_SHOTGUN',
         'OffenseFormation_SINGLEBACK',
         'OffenseFormation_WILDCAT',
         'OffenseFormation_I_FORM']
    for col in expected_columns:
        if not col in df.columns:
            df[col] = 0


    df['GameClock'] = df['GameClock'].apply(str_to_seconds)

    df['PlayerHeight'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
    df['PlayerBMI'] = 703*(df['PlayerWeight']/(df['PlayerHeight'])**2)

    df['TimeHandoff'] = pd.to_datetime(df['TimeHandoff'], utc=True)
    df['TimeSnap'] = pd.to_datetime(df['TimeSnap'], utc=True)
    df['TimeDelta'] = (df['TimeHandoff']-df['TimeSnap']).apply(lambda x: x.total_seconds())
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    df['PlayerBirthDate'] = pd.to_datetime(df['PlayerBirthDate'], utc=True)

    seconds_in_year = 60*60*24*365.25
    df['PlayerAge'] = (df['TimeHandoff']-df['PlayerBirthDate']).apply(lambda x: x.total_seconds())/seconds_in_year
    df = df.drop(['TimeHandoff', 'TimeSnap', 'PlayerBirthDate'], axis=1)

    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    df['WindSpeed'] = df['WindSpeed'].apply(str_to_float)

    df = df.drop(['WindDirection'], axis=1)
    df['PlayDirection'] = df['PlayDirection'].apply(lambda x: x.strip() == 'right')
    df['IsHomeTeam'] = df['Team'].apply(lambda x: x.strip()=='home')



    df['GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(map_weather)

    df['IsRusher'] = df['NflId'] == df['NflIdRusher']
    df.drop(['NflId', 'NflIdRusher'], axis=1, inplace=True)

    df['X'] = df.apply(lambda row: row['X'] if row['PlayDirection'] else 120-row['X'], axis=1)
    df['Orientation'] = df.apply(lambda row: new_orientation(row['Orientation'], row['PlayDirection']), axis=1)
    df['Dir'] = df.apply(lambda row: new_orientation(row['Dir'], row['PlayDirection']), axis=1)
    
    df['YardsLeft'] = df.apply(lambda row: 100-row['YardLine'] if row['HomeField'] else row['YardLine'], axis=1)
    df['YardsLeft'] = df.apply(lambda row: row['YardsLeft'] if row['PlayDirection'] else 100-row['YardsLeft'], axis=1)
    

    # DefensePersonnel
    counts = []
    for i, val in df['DefensePersonnel'].str.split(',').iteritems():
        row = {'OL': 0, 'RB': 0, 'TE': 0, 'WR': 0, 'DL': 0, 'DB': 0, 'LB': 0, 'QB': 0}
        if val is np.NaN:
            counts.append({})
            continue
        for item in val:
            name, number = item.strip().split(' ')[::-1]
            row[name] = int(number)
        counts.append(row)
    defense_presonell_df = pd.DataFrame(counts)
    defense_presonell_df.columns = ['defense_'+x for x in defense_presonell_df.columns]
    defense_presonell_df = defense_presonell_df.fillna(0).astype(int)
    defense_presonell_df.index = df.index
    df = pd.concat([df.drop(['DefensePersonnel'], axis=1), defense_presonell_df], axis=1)


    # OffensePersonnel
    counts = []
    for i, val in df['OffensePersonnel'].str.split(',').iteritems():
        row = {'OL': 0, 'RB': 0, 'TE': 0, 'WR': 0, 'DL': 0, 'DB': 0, 'LB': 0, 'QB': 0}
        if val is np.NaN:
            counts.append({})
            continue
        for item in val:
            name, number = item.strip().split(' ')[::-1]
            row[name] = int(number)
        counts.append(row)
    offense_personnel_df = pd.DataFrame(counts)
    offense_personnel_df.columns = ['offense_'+x for x in offense_personnel_df.columns]
    offense_personnel_df = offense_personnel_df.fillna(0).astype(int)
    offense_personnel_df.index = df.index
    df = pd.concat([df.drop(['OffensePersonnel'], axis=1), offense_personnel_df], axis=1)
    

    df = sort_df(df)

    if fillna:
        df.fillna(-999, inplace=True)
    return df

def sort_df(df):
    df = df.sort_values(by=['PlayId', 'InPosession', 'IsRusher']).reset_index(drop=True)
    return df

def make_x(df):
    source_play_id = df['PlayId']

    cols_delete = ['GameId', 'PlayId', 'IsRusher', 'Team']
    df = df.drop(cols_delete, axis=1)

    # Fill nan

    # Text features
    text_cols = [
        'FieldPosition',
         'DisplayName',
         'PossessionTeam',
         'PlayerCollegeName',
         'Position',
         'HomeTeamAbbr',
         'VisitorTeamAbbr',
         'Stadium',
         'Location']
    df = df.drop(text_cols, axis=1)

    # Player features
    cols_player = ['X',
         'Y',
         'S',
         'A',
         'Dis',
         'Orientation',
         'Dir',
         'JerseyNumber',
         'PlayerHeight',
         'PlayerWeight',
         'PlayerBMI',
         'PlayerAge']

    all_cols_player = np.array([[f'pl{num}_'+x for x in cols_player] for num in range(1, 23)]).flatten()

    X = np.array(df[cols_player]).reshape(-1, len(cols_player)*22)

    play_id_index = source_play_id[::22]
    X_df = pd.DataFrame(X, columns=all_cols_player, index=play_id_index)

    assert df[cols_player].shape[0] == X_df.shape[0] * 22
    assert df[cols_player].shape[1] == X_df.shape[1] / 22

    # Play features
    cols_play = list(df.drop(cols_player+(['Yards'] if 'Yards' in df.columns else []), axis=1).columns)
    X_play_col = np.zeros(shape=(X.shape[0], len(cols_play)))
    for i, col in enumerate(cols_play):
        X_play_col[:, i] = df[col][::22]

    X_play_col_df = pd.DataFrame(X_play_col, columns=cols_play, index=play_id_index)
    assert X_df.shape[0] == X_play_col_df.shape[0]
    X_df = pd.concat([X_df, X_play_col_df], axis=1)

    assert X_df.shape[0] == source_play_id.drop_duplicates().count()
    return X_df

def make_y(X, df):
    y = np.zeros(shape=(X.shape[0], 199))
    for i, yard in enumerate(df['Yards'][::22]):
        y[i, yard+99:] = np.ones(shape=(1, 100-yard))
    return y



In [ ]:
df_train = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', dtype={'WindSpeed': 'object'})
# df_train = pd.read_csv('data/train.csv', dtype={'WindSpeed': 'object'})

In [ ]:
df_train_preprocessed = preprocess_features(df_train, fillna=True)

In [ ]:
X_train = make_x(df_train_preprocessed)
y_train = make_y(X_train, df_train_preprocessed)

In [ ]:
X_train.shape, y_train.shape

# NN

In [ ]:
N_FEATURES = X_train.shape[1]

In [ ]:
def iterate_minibatches(X, y, batchsize):
    X = torch.as_tensor(X)
    y = torch.as_tensor(y)
    indices = np.random.permutation(np.arange(len(X)))
    for start in range(0, len(indices), batchsize):
        ix = indices[start: start + batchsize]
        yield X[ix], y[ix]

In [ ]:
DEVICE = 'cpu'
BATCH_SIZE = 32
LEARNING_RATE = 0.002
NUM_EPOCHS = 150
EARLY_STOP_AFTER = 10
WEIGHT_DECAY = 0.0002
SCHEDULER_PATIENCE = 5
L1_LOSS_WEIGHT = 0.001

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(N_FEATURES, N_FEATURES),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(N_FEATURES),
            nn.Linear(N_FEATURES, N_FEATURES),
            nn.ReLU(),
            nn.BatchNorm1d(N_FEATURES),
            nn.Linear(N_FEATURES, N_FEATURES),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(N_FEATURES),
        )
        self.head_classifier = nn.Sequential(
            nn.Linear(N_FEATURES, 199),
            nn.Softmax()
        )
        self.head_regressor = nn.Sequential(
            nn.Linear(N_FEATURES, 1),
        )

    def forward(self, x_batch):
        latent_out = self.seq.forward(x_batch)
        return self.head_classifier(latent_out), self.head_regressor(latent_out)

    def loss(self, x_batch, y_batch):
        y_batch = torch.as_tensor(y_batch).double().to(device=DEVICE)
        y_pdf, y_yard = self(x_batch)
        loss_crps = crps_loss(y_batch, y_pdf)
        
        y_yard = y_yard.flatten().double()
        y_batch_yard = cdf_to_yard_torch(y_batch).double()
        loss_mae = torch.mean(torch.abs(y_batch_yard-y_yard))
        return loss_crps + loss_mae*L1_LOSS_WEIGHT

    def predict_pdf(self, x_batch):
        y_pdf, y_yard = self(x_batch)
        return y_pdf
    
    def predict_cdf(self, x_batch):
        y_pred_pdf = self.predict_pdf(x_batch)
        y_pred_cdf = torch.cumsum(y_pred_pdf, dim=1)
        y_pred_cdf = torch.clamp(y_pred_cdf, 0, 1)
        return y_pred_cdf

model = NeuralNet()
model = model.double().to(device=DEVICE)

# Train utils

In [ ]:
def train(model, opt, scheduler, X_train, y_train, X_val, y_val, early_stop_patience, batch_size, n_epochs):
    train_loss = []
    train_loss_stds = []
    val_accuracy = []
    val_accuracy_stds = []

    best_val_acc = None
    best_model = None
    patience = 0

    for epoch in range(n_epochs):
        try:
            opt.zero_grad()

            start_time = time.time()
            model.train(True)
            epoch_loss = []

            for X_batch, y_batch in iter(iterate_minibatches(X_train, y_train, batch_size)):
                loss = model.loss(X_batch, y_batch)
                epoch_loss.append(float(loss.item()))
                loss.backward()
                opt.step()
                opt.zero_grad()

            train_loss.append(np.mean(epoch_loss))
            train_loss_stds.append(np.std(epoch_loss))
            model.train(False)

            epoch_val_acc = []
            for X_batch, y_batch in iter(iterate_minibatches(X_val, y_val, batch_size)): 
                y_val_pred_cdf = model.predict_cdf(X_batch).to(device=DEVICE)
                val_loss = crps_loss_cdf(y_batch, y_val_pred_cdf)
                epoch_val_acc.append(val_loss.item())
            val_acc = np.mean(epoch_val_acc)
            val_accuracy.append(val_acc)
            val_accuracy_stds.append(np.std(epoch_val_acc))


            scheduler.step(val_acc)

            # Then we print the results for this epoch:
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, n_epochs, time.time() - start_time))
            print("  training loss: \t{:.6f}".format(train_loss[-1]))
            print("  validation score: \t\t\t{:.6f}".format(val_accuracy[-1]))

            if best_val_acc is None or val_acc < best_val_acc:
                best_val_acc = val_acc
                patience = 0
                torch.save(model.state_dict(), 'best_model')
            else:
                patience += 1
                print(f'Validation score has not improved for {patience} epochs.')
            if patience >= early_stop_patience:
                print('Early stopping.')
                break
        except KeyboardInterrupt:
            break
    model.load_state_dict(torch.load('best_model'))
    model.train(False)

    history = {
        'train_loss': train_loss,
        'train_loss_stds': train_loss_stds,
        'val_accuracy': val_accuracy,
        'val_accuracy_stds': val_accuracy_stds,
    }
    
    return model, history

def train_nn_pipeline(model, X, y, early_stop_after=EARLY_STOP_AFTER, batch_size=BATCH_SIZE, n_epochs=NUM_EPOCHS):
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
    
    opt = torch.optim.Adamax(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', patience=SCHEDULER_PATIENCE, verbose=True, threshold=1e-6, eps=1e-9)

    model, train_history = train(model, opt, scheduler, X_train, y_train, X_val, y_val, early_stop_after, batch_size, n_epochs)
    
    return model, scaler, train_history

def nn_pipeline_predict(model, scaler, X):
    X = torch.as_tensor(scaler.transform(X)).to(device=DEVICE)
    y_pred = model.predict_cdf(X)
    return y_pred

# CV

In [ ]:
from sklearn.model_selection import RepeatedKFold

cv = RepeatedKFold(n_splits=5, n_repeats=1)

In [ ]:
def run_cv(X, y, cv, model_class, train_function):
    models, scalers, losses = [], [], []
    i = 0
    X = X.values
    for train_index, test_index in cv.split(X):
        print(f'Training model {i}')
        try:
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            model = model_class().double().to(device=DEVICE)
            model, scaler, history = train_nn_pipeline(model, X_train, y_train)
            y_pred_cdf_test = nn_pipeline_predict(model, scaler, X_test)
            y_test = torch.as_tensor(y_test).to(device=DEVICE)
            test_loss = crps_torch(y_test, y_pred_cdf_test).mean()
            losses.append(test_loss.cpu().data.item())
            models.append(model)
            scalers.append(scaler)
            i+=1
        except KeyboardInterrupt:
            break
    return models, scalers, losses

def cv_predict(models, scalers, X):
    X = X.values
    predicted_cdfs = []
    for model, scaler in zip(models, scalers):
        X_prep = torch.as_tensor(scaler.transform(X)).to(device=DEVICE)
        predicted_cdfs.append(model.predict_cdf(X_prep).cpu().data.numpy())
    return np.mean(predicted_cdfs, axis=0) 


In [ ]:
models, scalers, losses = run_cv(X_train, y_train, cv, NeuralNet, train_nn_pipeline)

# SUBMIT

In [ ]:
env = nflrush.make_env()

In [ ]:
def get_predictions(df_test, y_test, model):
    X = make_x(preprocess_features(df_test, fillna=True))
    y_pred = cv_predict(models, scalers, X)
    return y_pred

In [ ]:
# preprocess_features(df_test).columns

In [ ]:
# set(make_x(df_train_preprocessed).columns).difference(set(make_x(preprocess_features(df_test, fillna=-999))))

In [ ]:
# set(make_x(preprocess_features(df_test, fillna=True)).columns).difference(set(make_x(df_train_preprocessed).columns))

In [ ]:
iter_test = env.iter_test()

In [ ]:
for df_test, y_cdf_test in tqdm.tqdm(iter_test):
    y_pred = get_predictions(df_test, y_cdf_test, model)
    env.predict(pd.DataFrame(data=y_pred,columns=y_cdf_test.columns))

In [ ]:
env.write_submission_file()